# Validate candlestick patterns

## Import relevant libraries

In [1]:
from src import utils as u
import pandas as pd
import os
import numpy as np
import datetime
from configparser import ConfigParser

## Set some 'global' parameters

In [2]:
configur = ConfigParser()
print (configur.read('../config_files/config_candlestick_patterns.ini'))
#print ("Sections : ", configur.sections())

['../config_files/config_candlestick_patterns.ini']


## Get and process the data

### Method that executes the process (run)

In [3]:
def run():
    try:
        # Parameters
        list_asset_ticket = ["BTCUSDT", "ETHUSDT", "BNBUSDT"]
        list_timestamp = ["1d", "1h", "30m", "15m", "5m"]
        start_date = "1 Jan, 2017"
        end_date = "31 Dec, 2022"
        window_size = configur.getint('candlestick_patterns', 'window_size')
        slope_size = configur.getint('candlestick_patterns', 'slope_size')

        for asset_ticket in list_asset_ticket:
            for timestamp in list_timestamp:

                # Data file parameters
                input_data_path = '../../data/10_candlesticks_signals_raw'
                input_data_filename = "binance" + \
                                "_" + asset_ticket + \
                                "_" + timestamp + \
                                "_from_" + datetime.datetime.strptime(start_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                                "_to_" + datetime.datetime.strptime(end_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                                "_candlesticks_signals_raw"
                input_data_extension = ".csv"
                full_path_input_data = os.path.join(input_data_path, input_data_filename + input_data_extension)

                df = pd.read_csv(full_path_input_data)

                # Set the index on the dataframe
                df['date_index'] = df['date']
                df.set_index('date_index', inplace=True)

                # Set some parameters
                window_size = configur.getint('candlestick_patterns', 'window_size')
                slope_size = configur.getint('candlestick_patterns', 'slope_size')

                # Get the dates from the bullish candlesticks patterns
                list_dates_invertedhammer = df[df['CDLINVERTEDHAMMER'] == 100]['date'].to_list()
                list_dates_hammer = df[df['CDLHAMMER'] == 100]['date'].to_list()
                list_dates_piercing = df[df['CDLPIERCING'] == 100]['date'].to_list()
                list_dates_morningstar = df[df['CDLMORNINGSTAR'] == 100]['date'].to_list()
                list_dates_bullishengulfing = df[df['CDLENGULFINGBULLISH'] == 100]['date'].to_list()

                # Get all dates
                list_all_dates = df['date'].to_list()

                # Run the checks to identify the real candlesticks with reversal
                list_validate_dates_invertedhammer = []
                list_new_sign_invertedhammer = []
                list_sign_invertedhammer = [0] * len(list_all_dates)
                for date in list_dates_invertedhammer:
                    i = u.check_intensity_trend(df, date, window_size, slope_size)
                    if(i >= 0):
                        list_validate_dates_invertedhammer.append(date)
                        list_new_sign_invertedhammer.append(i*100)
                        list_sign_invertedhammer[list_all_dates.index(date)] = i*100


                list_validate_dates_hammer = []
                list_new_sign_hammer = []
                list_sign_hammer = [0] * len(list_all_dates)
                for date in list_dates_hammer:
                    i = u.check_intensity_trend(df, date, window_size, slope_size)
                    if(i >= 0):    
                        list_validate_dates_hammer.append(date)
                        list_new_sign_hammer.append(i*100)
                        list_sign_hammer[list_all_dates.index(date)] = i*100

                list_validate_dates_piercing = []
                list_new_sign_piercing = []
                list_sign_piercing = [0] * len(list_all_dates)
                for date in list_dates_piercing:
                    i = u.check_intensity_trend(df, date, window_size, slope_size)
                    if(i >= 0):
                        list_validate_dates_piercing.append(date)
                        list_new_sign_piercing.append(i*100)
                        list_sign_piercing[list_all_dates.index(date)] = i*100

                list_validate_dates_morningstar = []
                list_new_sign_morningstar = []
                list_sign_morningstar = [0] * len(list_all_dates)
                for date in list_dates_morningstar:
                    i = u.check_intensity_trend(df, date, window_size, slope_size)
                    if(i >= 0):
                        list_validate_dates_morningstar.append(date)
                        list_new_sign_morningstar.append(i*100)
                        list_sign_morningstar[list_all_dates.index(date)] = i*100

                list_validate_dates_bullishengulfing = []
                list_new_sign_bullishengulfing = []
                list_sign_bullishengulfing = [0] * len(list_all_dates)
                for date in list_dates_bullishengulfing:
                    i = u.check_intensity_trend(df, date, window_size, slope_size)
                    # print(f'date:{date}|i:{i}')
                    if(i >= 0):
                        list_validate_dates_bullishengulfing.append(date)
                        list_new_sign_bullishengulfing.append(i*100)
                        list_sign_bullishengulfing[list_all_dates.index(date)] = i*100

                # Get the dates from the bearish candlesticks patterns
                list_dates_shootingstar = df[df['CDLSHOOTINGSTAR'] == -100]['date'].to_list()
                list_dates_hangingman = df[df['CDLHANGINGMAN'] == -100]['date'].to_list()
                list_dates_darkcloudcover = df[df['CDLDARKCLOUDCOVER'] == -100]['date'].to_list()
                list_dates_eveningstar = df[df['CDLEVENINGSTAR'] == -100]['date'].to_list()
                list_dates_bearishengulfing = df[df['CDLENGULFINGBEARISH'] == -100]['date'].to_list()

                # Run the checks to identify the real candlesticks with reversal
                list_validate_dates_shootingstar = []
                list_new_sign_shootingstar = []
                list_sign_shootingstar = [0] * len(list_all_dates)
                for date in list_dates_shootingstar:
                    i = u.check_intensity_trend(df, date, window_size, slope_size)
                    if (i<=0):
                        list_validate_dates_shootingstar.append(date)
                        list_new_sign_shootingstar.append(i*100)
                        list_sign_shootingstar[list_all_dates.index(date)] = i*100


                list_validate_dates_hangingman = []
                list_new_sign_hangingman = []
                list_sign_hangingman = [0] * len(list_all_dates)
                for date in list_dates_hangingman:
                    i = u.check_intensity_trend(df, date, window_size, slope_size)
                    if (i<=0):
                        list_validate_dates_hangingman.append(date)
                        list_new_sign_hangingman.append(i*100)
                        list_sign_hangingman[list_all_dates.index(date)] = i*100

                list_validate_dates_darkcloudcover = []
                list_new_sign_darkcloudcover = []
                list_sign_darkcloudcover = [0] * len(list_all_dates)
                for date in list_dates_darkcloudcover:
                    i = u.check_intensity_trend(df, date, window_size, slope_size)
                    if (i<=0):
                        list_validate_dates_darkcloudcover.append(date)
                        list_new_sign_darkcloudcover.append(i*100)
                        list_sign_darkcloudcover[list_all_dates.index(date)] = i*100  

                list_validate_dates_eveningstar = []
                list_new_sign_eveningstar = []
                list_sign_eveningstar = [0] * len(list_all_dates)
                for date in list_dates_eveningstar:
                    i = u.check_intensity_trend(df, date, window_size, slope_size)
                    if (i<=0):
                        list_validate_dates_eveningstar.append(date)
                        list_new_sign_eveningstar.append(i*100)
                        list_sign_eveningstar[list_all_dates.index(date)] = i*100

                list_validate_dates_bearishengulfing = []
                list_new_sign_bearishengulfing = []
                list_sign_bearishengulfing = [0] * len(list_all_dates)
                for date in list_dates_bearishengulfing:
                    i = u.check_intensity_trend(df, date, window_size, slope_size)
                    if (i<=0):
                        list_validate_dates_bearishengulfing.append(date)
                        list_new_sign_bearishengulfing.append(i*100)
                        list_sign_bearishengulfing[list_all_dates.index(date)] = i*100

                # Create the data with the validation results

                list_new_column_name_pos = ['CDLINVERTEDHAMMER_NEW', 'CDLHAMMER_NEW', 'CDLPIERCING_NEW', 'CDLMORNINGSTAR_NEW', 'CDLENGULFINGBULLISH_NEW']
                list_new_column_name_neg = ['CDLSHOOTINGSTAR_NEW', 'CDLHANGINGMAN_NEW', 'CDLDARKCLOUDCOVER_NEW', 'CDLEVENINGSTAR_NEW', 'CDLENGULFINGBEARISH_NEW']

                list_of_list_valid_dates_pos = [list_validate_dates_invertedhammer, list_validate_dates_hammer, list_validate_dates_piercing, list_validate_dates_morningstar, list_validate_dates_bullishengulfing]
                list_of_list_valid_dates_neg = [list_validate_dates_shootingstar, list_validate_dates_hangingman, list_validate_dates_darkcloudcover, list_validate_dates_eveningstar, list_validate_dates_bearishengulfing]

                list_of_list_sign_pos = [list_sign_invertedhammer, list_sign_hammer, list_sign_piercing, list_sign_morningstar, list_sign_bullishengulfing]
                list_of_list_sign_neg = [list_sign_shootingstar, list_sign_hangingman, list_sign_darkcloudcover, list_sign_eveningstar, list_sign_bearishengulfing]

                for i in range(0, len(list_new_column_name_pos)):
                    df[list_new_column_name_pos[i]] = np.where(df['date'].isin(list_of_list_valid_dates_pos[i]), list_of_list_sign_pos[i], 0)

                for i in range(0, len(list_new_column_name_neg)):
                    df[list_new_column_name_neg[i]] = np.where(df['date'].isin(list_of_list_valid_dates_neg[i]), list_of_list_sign_neg[i], 0)

                # Export the data
                export_path = "../../data/20_candlesticks_signals_processed"
                export_filename = "binance" + \
                                "_" + asset_ticket + \
                                "_" + timestamp + \
                                "_from_" + datetime.datetime.strptime(start_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                                "_to_" + datetime.datetime.strptime(end_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                                "_candlesticks_signals_processed"
                export_extension = ".csv"
                full_export_path = os.path.join(export_path, export_filename + export_extension)            

                df.to_csv(full_export_path, index=False)

    except Exception as e:
        print(e)

### Run!

In [4]:
run()